# Generic Conditional Laws for Random-Fields - via:

## Universal $\mathcal{P}_1(\mathbb{R})$-Deep Neural Model $\mathcal{NN}_{1_{\mathbb{R}^n},\mathcal{D}}^{\sigma:\star}$.

---

By: [Anastasis Kratsios](https://people.math.ethz.ch/~kratsioa/) - 2021.

---

In [1]:
# Software/Hardware Testing or Real-Deal?
trial_run = False

---
# Training Algorithm:
---
- Random $\delta$-bounded partition on input space,
- Train deep classifier on infered classes.
---
---
---
## Notes - Why the procedure is so computationally efficient?
---
 - The sample barycenters do not require us to solve for any new Wasserstein-1 Barycenters; which is much more computationally costly,
 - Our training procedure never back-propages through $\mathcal{W}_1$ since steps 2 and 3 are full-decoupled.  Therefore, training our deep classifier is (comparatively) cheap since it takes values in the standard $N$-simplex.

---

## Load Auxiliaries

In [2]:
# Load Packages/Modules
exec(open('Init_Dump.py').read())
# Load Hyper-parameter Grid
exec(open('CV_Grid.py').read())
# Load Helper Function(s)
exec(open('Helper_Functions.py').read())
# Import time separately
import time


# load dataset
results_path = "./outputs/models/"
results_tables_path = "./outputs/results/"
raw_data_path_folder = "./inputs/raw/"
data_path_folder = "./inputs/data/"


### Set Seed
random.seed(2021)
np.random.seed(2021)
tf.random.set_seed(2021)

Using TensorFlow backend.


Deep Feature Builder - Ready
Deep Classifier - Ready


## Meta-Parameters

### Simulation

#### Grid Hyperparameter(s)
- Ratio $\frac{\text{Testing Datasize}}{\text{Training Datasize}}$.
- Number of Training Points to Generate

In [3]:
train_test_ratio = .2
N_train_size = 10**2

Monte-Carlo Paramters

In [4]:
## Monte-Carlo
N_Euler_Maruyama_Steps = 2
N_Monte_Carlo_Samples = 10**3

# End times for Time-Grid
T_end = 1

Initial radis of $\delta$-bounded random partition of $\mathcal{X}$!

In [5]:
# Hyper-parameters of Cover
delta = 0.001
Proportion_per_cluster = .1

**Note**: Setting *N_Quantizers_to_parameterize* prevents any barycenters and sub-sampling.

In [6]:
trial_run = True

## Simulation from Rough Neural-SDE
Simulate via Euler-M method from:
$$ 
X_T^x = x + \int_0^T \alpha(s,X_s^x)ds + \int_0^T\beta(s,X_s^s)dB_s^H.
$$
We seek to learn $\mathbb{P}\left(A^{\top}X_1^x\mid X_0^x = x\right)$, where $A^{\top}$ is an unknown $1\times d$-matrix.  

## Problem Dimension

In [7]:
problem_dim = 5

width = 10**2

### Drift

In [8]:
W_2a = np.random.uniform(size=np.array([problem_dim,width]),low=-.5,high=.5)
W_1a = np.random.uniform(size=np.array([width,problem_dim]),low=-.5,high=.5)
def alpha(t,x):
    x_internal = x.reshape(-1,)
#     x_internal = np.matmul(W_1a,x_internal)
#     x_internal = np.matmul(W_2a,np.cos(x_internal))
    x_internal = 0.01*x_internal
    return x_internal

### Volatility

In [9]:
W_2b = np.random.uniform(size=np.array([(problem_dim**2),width]),low=-.5,high=.5)
W_1b = np.random.uniform(size=np.array([width,problem_dim]),low=-.5,high=.5)
def beta(t,x):
#     x_internal = x.reshape(-1,)
#     x_internal = np.matmul(W_1b,x_internal)
#     x_internal = np.matmul(W_2b,np.maximum(0,np.array(x_internal)))
#     x_internal = x_internal.reshape([problem_dim,problem_dim])
#     # Ensure PSD
#     x_internal = np.matmul(x_internal,x_internal.T)
    x_internal = 0.001*np.diag(np.ones(problem_dim))
    return x_internal

### Roughness Meta-parameters
 - Roughness is $H$,
 - Ratio_fBM_to_typical_vol is $\eta$.

In [10]:
Rougness = 0.5 # Hurst Parameter

## Initialize Simulator

#### Initialize Unknown Matrix

In [11]:
A_unkown = np.random.normal(size=np.array([problem_dim,1]))

Simulate using Euler-Maruyama + Monte-Carlo

In [12]:
###################################
# Define Simulator Data Generator #
###################################
def Euler_Maruyama_Generator(x_0,
                             N_Euler_Maruyama_Steps = 10,
                             N_Monte_Carlo_Samples = 100,
                             T_begin = 0,
                             T_end = T_end,
                             Hurst = 0.1): 
    #----------------------------#    
    # DEFINE INTERNAL PARAMETERS #
    #----------------------------#
    # Internal Initialization(s)
    ## Initialize current state
    n_sample = 0
    ## Initialize Incriments
    dt = (T_end-T_begin)/N_Euler_Maruyama_Steps
    sqrt_dt = np.sqrt(dt)
    # Get Dimension
    dim_x_0 = len(x_0)

    #-----------------------------#    
    # Generate Monte-Carlo Sample #
    #-----------------------------#
    for n_sample in range(N_Monte_Carlo_Samples):
        # Initialize Current State 
        X_current = x_0
        # Generate roughness
        for fBM_path_i in range(dim_x_0):
            sigma_rough_loop = FBM(n=N_Euler_Maruyama_Steps, 
                                   hurst=Hurst, 
                                   length=1, 
                                   method='daviesharte').fbm().reshape(1,-1)
            if fBM_path_i == 0:
                sigma_rough = sigma_rough_loop
            else:
                sigma_rough = np.append(sigma_rough,sigma_rough_loop,axis=0)

    #     Perform Euler-Maruyama Simulation
        for t in range((sigma_rough.shape[1]-1)):
            # Update Internal Parameters
            ## Get Current Time
            t_current = t*((T_end - T_begin)/N_Euler_Maruyama_Steps)

            # Update Generated Path
            ## Generate Current State-Update Components
            drift_t = alpha(t_current,X_current.reshape(-1,))*dt
            vol_t = beta(t_current,X_current)
            BH_t = (sigma_rough[:,(1+t)])
            ## Compute Update
            X_current = X_current + drift_t + (np.matmul(vol_t,BH_t))       

        # Update Empirical Measure
        X_current = X_current.reshape(-1,1)
        if n_sample ==0:
            X_T_Empirical = X_current
        else:
            X_T_Empirical = np.append(X_T_Empirical,X_current,axis=-1)

    # Reshape
    X_T_Empirical = X_T_Empirical.T
    
    # Get Output
    Y_Empirical = np.matmul(X_T_Empirical,A_unkown)
    
    
    # Add Stationary Uniform Noise
#     if uniform_noise>0:
#         X_T_Empirical = X_T_Empirical #+ np.random.uniform(low=-uniform_noise,high=uniform_noise,size=X_T_Empirical.shape())
    return Y_Empirical










################################
# Define Output Data Generator #
################################
def Euler_Maruyama_simulator(Grid_in,
                             N_Monte_Carlo_Samples = N_Monte_Carlo_Samples,
                             Rougness = Rougness,
                             N_Euler_Maruyama_Steps = N_Euler_Maruyama_Steps):
    #----------------------------#
    ## Generate Data Using Grid ##
    #----------------------------#
    # Internal Parameters
    N_Grid_Instances = Grid_in.shape[0]
    # Initializations
    measure_weights = np.ones(N_Monte_Carlo_Samples)/N_Monte_Carlo_Samples
    measures_locations_list_internal = []
    measures_weights_list_internal = []


    # Perform Euler-Maruyama distritization + Monte-Carlo Sampling.
    #----------------------------------------------------------------------------------------------#
    # Perform Monte-Carlo Data Generation
    for i in tqdm(range(N_Grid_Instances)):
        x_loop = Grid_in[i,]
        # Simulate Paths
        paths_loop = Euler_Maruyama_Generator(x_0=x_loop,
                                              N_Euler_Maruyama_Steps = N_Euler_Maruyama_Steps,
                                              N_Monte_Carlo_Samples = N_Monte_Carlo_Samples,
                                              T_begin = 0,
                                              T_end = T_end,
                                              Hurst = Rougness)

        # Map numpy to list
        measures_locations_loop = paths_loop#.tolist()

        # Append to List
        measures_locations_list_internal.append(measures_locations_loop)
        measures_weights_list_internal.append(measure_weights)


    return measures_locations_list_internal, measures_weights_list_internal

## Initialize Data

In [13]:
N_test_size = int(np.round(N_train_size*train_test_ratio,0))

### Initialize Training Data (Inputs)

In [14]:
# Get Training Set
X_train = np.random.uniform(size=np.array([N_train_size,problem_dim]),low=.5,high=1.5)

# Get Testing Set
test_set_indices = np.random.choice(range(X_train.shape[0]),N_test_size)
X_test = X_train[test_set_indices,]
X_test = X_test + np.random.uniform(low=-(delta/np.sqrt(problem_dim)), 
                                    high = -(delta/np.sqrt(problem_dim)),
                                    size = X_test.shape)

### Initialize Training Data (Outputs)

#### Generate Output Data

In [15]:
# Get Training Data
## Timer
train_DATA_MC = time.time()
## Do: Simulation
Y_train_locations,Y_train_weights = Euler_Maruyama_simulator(X_train)
# X_train = pd.DataFrame(X_train)
## END: TIMER
train_DATA_MC = time.time() - train_DATA_MC

# Get Testing Data
## Timer
test_DATA_MC = time.time()
## Do: Simulation
Y_test_locations,Y_test_weights = Euler_Maruyama_simulator(X_test)
# X_test = pd.DataFrame(X_test)
## END: TIMER
test_DATA_MC = time.time() - test_DATA_MC

100%|██████████| 20/20 [00:04<00:00,  4.53it/s]


# Train Model
#### Start Timer

In [16]:
# Start Timer
Type_A_timer_Begin = time.time()

### $\omega^{-1}(\epsilon)$-Bounded Random Partitioner
Generates a bounded random partition, in the sense of: [Geometry, Flows, and Graph-Partitioning Algorithms](https://cacm.acm.org/magazines/2008/10/515-geometry-flows-and-graph-partitioning-algorithms/fulltext?mobile=false), [A. Naor et al.](https://link.springer.com/article/10.1007/s00222-004-0400-5), as implemented in [Learning Sub-Patterns in Piece-Wise Continuous Functions](https://arxiv.org/abs/2010.15571).

In [17]:
# Initialize #
#------------#
X_training_remaining = np.copy(X_train)
possible_indices_for_center = np.repeat(True,X_training_remaining.shape[0])
indexing_set = np.array(range(X_training_remaining.shape[0]))

# Tweak Radius of Balls
# Note: To avoid "too small" of a delta
dist_matrix = (distance_matrix(X_training_remaining[np.random.choice(indexing_set,max(1,round(X_training_remaining.shape[0]*.5)))],X_training_remaining[np.random.choice(indexing_set,max(1,round(X_training_remaining.shape[0]*.5)))]))
dist_matrix = (dist_matrix[np.logical_not(dist_matrix == 0)]).reshape(-1,)
delta = np.maximum(delta,np.quantile(dist_matrix,Proportion_per_cluster))
# delta = np.quantile(dist_matrix,np.minimum(0.01,N_train_size/(((problem_dim)/(np.sqrt(1+problem_dim)*delta/4))**2)))


# Build #
#-------#
while np.max(possible_indices_for_center)==True:  
    # Randomize Radius
    delta_loop = np.random.uniform(low=0,high=delta,size=1)[0]

    # Get Random Center
    random_center_loop_index = np.random.choice(indexing_set[possible_indices_for_center])
    random_center_loop = (X_training_remaining[random_center_loop_index]).reshape([1,-1])

    # Get Distances To Current Center
    dist_mat_loop = np.sqrt(np.sum((random_center_loop-X_training_remaining)**2,axis=1))
    ## Indentify which must lie in cluster but not counting previously removed elements
    indices_cluster_loop = (dist_mat_loop<delta_loop)*possible_indices_for_center

    # Update(s)
    ## Outputs
    if np.min(possible_indices_for_center)==True:
        ## Initialize Classes
        Train_classes = (indices_cluster_loop*1).reshape(-1,1)
        ## PointMasses Center
        Masses = int(random_center_loop_index)
        # INITIALIZE: Barycenters Array
        Barycenters_Array = (Y_train_locations[int(random_center_loop_index)]).reshape(-1,)
    else:
        ## Update Classes
        Train_classes = np.append(Train_classes,(indices_cluster_loop*1).reshape(-1,1),axis=-1)
        ## PointMasses Center
        Masses = np.append(Masses,random_center_loop_index)
        # UPDATE: Populate Barycenters Array
        Barycenters_Array = np.append(Barycenters_Array,(Y_train_locations[int(random_center_loop_index)]).reshape(-1,),axis=-1)
    ## Remove Clusterd Centers from Current Dataset
    possible_indices_for_center = np.logical_not(indices_cluster_loop)*possible_indices_for_center

# Format Training Classes
Train_classes = pd.DataFrame(Train_classes)

# Get Number of Classes
N_Quantizers_to_parameterize = Train_classes.shape[1]

# Update User #
print("---------------")
print("==============---------------------------------------------------=============")
print("Number of Classes/measures Which Were Produced:", N_Quantizers_to_parameterize)
print("==============---------------------------------------------------=============")
print("===========================")
print("Training Classes Produced:")
print("===========================")
print(Train_classes)
print("---------------")

---------------
==============---------------------------------------------------=============
Number of Classes/measures Which Were Produced: 58
==============---------------------------------------------------=============
Training Classes Produced:
    0   1   2   3   4   5   6   7   8   9   ...  48  49  50  51  52  53  54  \
0    0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   1   
1    0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
2    0   0   1   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
3    0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
4    0   0   0   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..   
95   0   0   0   0   0   0   0   0   0   0  ...   0   0   1   0   0   0   0   
96   0   0   0   0   0   0   1   0   0   0  ...   0   0   0   0   0   0   0   
97   0   0   0   0   0   0   0   0   

---

### Train Deep Classifier

In this step, we train a deep (feed-forward) classifier:
$$
\hat{f}\triangleq \operatorname{Softmax}_N\circ W_J\circ \sigma \bullet \dots \sigma \bullet W_1,
$$
to identify which barycenter we are closest to.

Re-Load Grid and Redefine Relevant Input/Output dimensions in dictionary.

#### Train Deep Classifier

Re-Load Packages and CV Grid

In [18]:
# Re-Load Hyper-parameter Grid
exec(open('CV_Grid.py').read())
# Re-Load Classifier Function(s)
exec(open('Helper_Functions.py').read())

Deep Feature Builder - Ready
Deep Classifier - Ready


Train Deep Classifier

In [19]:
print("==========================================")
print("Training Classifer Portion of Type-A Model")
print("==========================================")

# Redefine (Dimension-related) Elements of Grid
param_grid_Deep_Classifier['input_dim'] = [problem_dim]
param_grid_Deep_Classifier['output_dim'] = [N_Quantizers_to_parameterize]

# Train simple deep classifier
predicted_classes_train, predicted_classes_test, N_params_deep_classifier, timer_output = build_simple_deep_classifier(n_folds = CV_folds, 
                                                                                                        n_jobs = n_jobs, 
                                                                                                        n_iter = n_iter, 
                                                                                                        param_grid_in=param_grid_Deep_Classifier, 
                                                                                                        X_train = X_train, 
                                                                                                        y_train = Train_classes,
                                                                                                        X_test = X_test)

print("===============================================")
print("Training Classifer Portion of Type Model: Done!")
print("===============================================")

Training Classifer Portion of Type-A Model
Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:   19.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:   19.1s finished


Epoch 1/600
4/4 [==============================] - 0s 7ms/step - loss: 4.0124 - accuracy: 0.1200
Epoch 2/600
4/4 [==============================] - 0s 7ms/step - loss: 3.7741 - accuracy: 0.1700
Epoch 3/600
4/4 [==============================] - 0s 8ms/step - loss: 3.5523 - accuracy: 0.1700
Epoch 4/600
4/4 [==============================] - 0s 7ms/step - loss: 3.5339 - accuracy: 0.1700
Epoch 5/600
4/4 [==============================] - 0s 7ms/step - loss: 3.4223 - accuracy: 0.2500
Epoch 6/600
4/4 [==============================] - 0s 8ms/step - loss: 3.3887 - accuracy: 0.3300
Epoch 7/600
4/4 [==============================] - 0s 9ms/step - loss: 3.2108 - accuracy: 0.3000
Epoch 8/600
4/4 [==============================] - 0s 8ms/step - loss: 3.0946 - accuracy: 0.2800
Epoch 9/600
4/4 [==============================] - 0s 9ms/step - loss: 2.9292 - accuracy: 0.3400
Epoch 10/600
4/4 [==============================] - 0s 8ms/step - loss: 2.7448 - accuracy: 0.3600
Epoch 11/600
4/4 [===========

4/4 [==============================] - 0s 7ms/step - loss: 0.0106 - accuracy: 1.0000
Epoch 85/600
4/4 [==============================] - 0s 7ms/step - loss: 0.0114 - accuracy: 1.0000
Epoch 86/600
4/4 [==============================] - 0s 6ms/step - loss: 0.0099 - accuracy: 1.0000
Epoch 87/600
4/4 [==============================] - 0s 7ms/step - loss: 0.0086 - accuracy: 1.0000
Epoch 88/600
4/4 [==============================] - 0s 9ms/step - loss: 0.0080 - accuracy: 1.0000
Epoch 89/600
4/4 [==============================] - 0s 6ms/step - loss: 0.0075 - accuracy: 1.0000
Epoch 90/600
4/4 [==============================] - 0s 6ms/step - loss: 0.0073 - accuracy: 1.0000
Epoch 91/600
4/4 [==============================] - 0s 7ms/step - loss: 0.0070 - accuracy: 1.0000
Epoch 92/600
4/4 [==============================] - 0s 6ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 93/600
4/4 [==============================] - 0s 7ms/step - loss: 0.0068 - accuracy: 1.0000
Epoch 94/600
4/4 [===============

4/4 [==============================] - 0s 6ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 168/600
4/4 [==============================] - 0s 7ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 169/600
4/4 [==============================] - 0s 6ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 170/600
4/4 [==============================] - 0s 8ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 171/600
4/4 [==============================] - 0s 6ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 172/600
4/4 [==============================] - 0s 7ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 173/600
4/4 [==============================] - 0s 6ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 174/600
4/4 [==============================] - 0s 7ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 175/600
4/4 [==============================] - 0s 6ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 176/600
4/4 [==============================] - 0s 8ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 177/600
4/4 [=====

4/4 [==============================] - 0s 7ms/step - loss: 7.3352e-04 - accuracy: 1.0000
Epoch 249/600
4/4 [==============================] - 0s 6ms/step - loss: 6.9752e-04 - accuracy: 1.0000
Epoch 250/600
4/4 [==============================] - 0s 7ms/step - loss: 6.8024e-04 - accuracy: 1.0000
Epoch 251/600
4/4 [==============================] - 0s 6ms/step - loss: 6.7459e-04 - accuracy: 1.0000
Epoch 252/600
4/4 [==============================] - 0s 7ms/step - loss: 7.0751e-04 - accuracy: 1.0000
Epoch 253/600
4/4 [==============================] - 0s 6ms/step - loss: 7.1064e-04 - accuracy: 1.0000
Epoch 254/600
4/4 [==============================] - 0s 7ms/step - loss: 6.7213e-04 - accuracy: 1.0000
Epoch 255/600
4/4 [==============================] - 0s 7ms/step - loss: 6.6004e-04 - accuracy: 1.0000
Epoch 256/600
4/4 [==============================] - 0s 7ms/step - loss: 6.9218e-04 - accuracy: 1.0000
Epoch 257/600
4/4 [==============================] - 0s 7ms/step - loss: 6.8095e-04 - a

4/4 [==============================] - 0s 7ms/step - loss: 3.7901e-04 - accuracy: 1.0000
Epoch 328/600
4/4 [==============================] - 0s 7ms/step - loss: 3.6615e-04 - accuracy: 1.0000
Epoch 329/600
4/4 [==============================] - 0s 8ms/step - loss: 3.6331e-04 - accuracy: 1.0000
Epoch 330/600
4/4 [==============================] - 0s 8ms/step - loss: 3.8630e-04 - accuracy: 1.0000
Epoch 331/600
4/4 [==============================] - 0s 7ms/step - loss: 3.9439e-04 - accuracy: 1.0000
Epoch 332/600
4/4 [==============================] - 0s 6ms/step - loss: 3.8655e-04 - accuracy: 1.0000
Epoch 333/600
4/4 [==============================] - 0s 7ms/step - loss: 3.6401e-04 - accuracy: 1.0000
Epoch 334/600
4/4 [==============================] - 0s 6ms/step - loss: 3.5394e-04 - accuracy: 1.0000
Epoch 335/600
4/4 [==============================] - 0s 7ms/step - loss: 3.5723e-04 - accuracy: 1.0000
Epoch 336/600
4/4 [==============================] - 0s 7ms/step - loss: 3.5820e-04 - a

4/4 [==============================] - 0s 6ms/step - loss: 2.1956e-04 - accuracy: 1.0000
Epoch 407/600
4/4 [==============================] - 0s 7ms/step - loss: 2.1674e-04 - accuracy: 1.0000
Epoch 408/600
4/4 [==============================] - 0s 6ms/step - loss: 2.1509e-04 - accuracy: 1.0000
Epoch 409/600
4/4 [==============================] - 0s 7ms/step - loss: 2.1420e-04 - accuracy: 1.0000
Epoch 410/600
4/4 [==============================] - 0s 6ms/step - loss: 2.1374e-04 - accuracy: 1.0000
Epoch 411/600
4/4 [==============================] - 0s 7ms/step - loss: 2.1221e-04 - accuracy: 1.0000
Epoch 412/600
4/4 [==============================] - 0s 7ms/step - loss: 2.1005e-04 - accuracy: 1.0000
Epoch 413/600
4/4 [==============================] - 0s 8ms/step - loss: 2.1041e-04 - accuracy: 1.0000
Epoch 414/600
4/4 [==============================] - 0s 7ms/step - loss: 2.1060e-04 - accuracy: 1.0000
Epoch 415/600
4/4 [==============================] - 0s 7ms/step - loss: 2.0718e-04 - a

4/4 [==============================] - 0s 8ms/step - loss: 1.4513e-04 - accuracy: 1.0000
Epoch 486/600
4/4 [==============================] - 0s 8ms/step - loss: 1.4464e-04 - accuracy: 1.0000
Epoch 487/600
4/4 [==============================] - 0s 7ms/step - loss: 1.4431e-04 - accuracy: 1.0000
Epoch 488/600
4/4 [==============================] - 0s 7ms/step - loss: 1.4372e-04 - accuracy: 1.0000
Epoch 489/600
4/4 [==============================] - 0s 7ms/step - loss: 1.4286e-04 - accuracy: 1.0000
Epoch 490/600
4/4 [==============================] - 0s 9ms/step - loss: 1.4164e-04 - accuracy: 1.0000
Epoch 491/600
4/4 [==============================] - 0s 7ms/step - loss: 1.4047e-04 - accuracy: 1.0000
Epoch 492/600
4/4 [==============================] - 0s 7ms/step - loss: 1.3985e-04 - accuracy: 1.0000
Epoch 493/600
4/4 [==============================] - 0s 6ms/step - loss: 1.3937e-04 - accuracy: 1.0000
Epoch 494/600
4/4 [==============================] - 0s 6ms/step - loss: 1.3867e-04 - a

4/4 [==============================] - 0s 6ms/step - loss: 1.0137e-04 - accuracy: 1.0000
Epoch 565/600
4/4 [==============================] - 0s 6ms/step - loss: 1.0040e-04 - accuracy: 1.0000
Epoch 566/600
4/4 [==============================] - 0s 7ms/step - loss: 1.0134e-04 - accuracy: 1.0000
Epoch 567/600
4/4 [==============================] - 0s 8ms/step - loss: 1.0066e-04 - accuracy: 1.0000
Epoch 568/600
4/4 [==============================] - 0s 9ms/step - loss: 1.0008e-04 - accuracy: 1.0000
Epoch 569/600
4/4 [==============================] - 0s 8ms/step - loss: 1.0003e-04 - accuracy: 1.0000
Epoch 570/600
4/4 [==============================] - 0s 6ms/step - loss: 9.9893e-05 - accuracy: 1.0000
Epoch 571/600
4/4 [==============================] - 0s 7ms/step - loss: 9.9063e-05 - accuracy: 1.0000
Epoch 572/600
4/4 [==============================] - 0s 6ms/step - loss: 9.7839e-05 - accuracy: 1.0000
Epoch 573/600
4/4 [==============================] - 0s 6ms/step - loss: 9.6551e-05 - a

#### Get Predicted Quantized Distributions
- Each *row* of "Predicted_Weights" is the $\beta\in \Delta_N$.
- Each *Column* of "Barycenters_Array" denotes the $x_1,\dots,x_N$ making up the points of the corresponding empirical measures.

In [20]:
# Format Weights
## Train
print("#---------------------------------------#")
print("Building Training Set (Regression): START")
print("#---------------------------------------#")
for i in tqdm(range((X_train.shape[0]))):
    for j in range(N_Quantizers_to_parameterize):
        b_loop = np.repeat(predicted_classes_train[i,j],N_Monte_Carlo_Samples)
        if j == 0:
            b = b_loop
        else:
            b = np.append(b,b_loop)
        b = b.reshape(-1,1)

    if i == 0:
        Predicted_Weights = b/np.sum(b)
    else:
        Predicted_Weights = np.append(Predicted_Weights,b,axis=-1)
Predicted_Weights = Predicted_Weights.T
print("#-------------------------------------#")
print("Building Training Set (Regression): END")
print("#-------------------------------------#")

## Test
print("#-------------------------------------#")
print("Building Test Set (Predictions): START")
print("#-------------------------------------#")
for i in tqdm(range((X_test.shape[0]))):
    for j in range(N_Quantizers_to_parameterize):
        b_loop = np.repeat(predicted_classes_test[i,j],N_Monte_Carlo_Samples)
        if j == 0:
            b = b_loop
        else:
            b = np.append(b,b_loop)
        b = b.reshape(-1,1)

    if i == 0:
        Predicted_Weights_test = b/np.sum(b)
    else:
        Predicted_Weights_test = np.append(Predicted_Weights_test,b,axis=-1)
Predicted_Weights_test = Predicted_Weights_test.T
print("#-----------------------------------#")
print("Building Test Set (Predictions): END")
print("#-----------------------------------#")

  0%|          | 0/100 [00:00<?, ?it/s]

#---------------------------------------#
Building Training Set (Regression): START
#---------------------------------------#


100%|██████████| 20/20 [00:00<00:00, 371.94it/s]

#-------------------------------------#
Building Training Set (Regression): END
#-------------------------------------#
#-------------------------------------#
Building Test Set (Predictions): START
#-------------------------------------#
#-----------------------------------#
Building Test Set (Predictions): END
#-----------------------------------#


#### Stop Timer

In [21]:
# Stop Timer
Type_A_timer_end = time.time()
# Compute Lapsed Time Needed For Training
Time_Lapse_Model_A = Type_A_timer_end - Type_A_timer_Begin

### Get Model Complexities

In [22]:
Model_Complexity = pd.DataFrame({"N_Centers":N_Quantizers_to_parameterize,
                                 "N_Q":N_Monte_Carlo_Samples,
                                 "N_Params":N_params_deep_classifier,
                                 "Training Time":Time_Lapse_Model_A,
                                 "T_Test/T_Test-MC": (timer_output/test_DATA_MC),
                                 "Time Test": timer_output,
                                 "Time EM-MC": test_DATA_MC},index=["Model_Complexity_metrics"])

pd.set_option('display.float_format', '{:.4E}'.format)
Model_Complexity.to_latex((results_tables_path+str("Roughness_")+str(Rougness)+"Latent_Width_NSDE"+str(width)+"Problemdimension"+str(problem_dim)+"__ModelComplexities.tex"))

## Get Moment Predictions

#### Write Predictions

### Training-Set Result(s): 

In [23]:
print("Building Training Set Performance Metrics")

# Initialize Wasserstein-1 Error Distribution
W1_errors = np.array([])
Mean_errors = np.array([])
Var_errors = np.array([])
Skewness_errors = np.array([])
Kurtosis_errors = np.array([])
predictions_mean = np.array([])
true_mean = np.array([])
#---------------------------------------------------------------------------------------------#

# Populate Error Distribution
for x_i in tqdm(range(X_train.shape[0])):    
    # Get Laws
    W1_loop = ot.emd2_1d(Barycenters_Array,
                         np.array(Y_train_locations[x_i]).reshape(-1,),
                         Predicted_Weights[x_i,].reshape(-1,),
                         (np.array(Y_train_locations[x_i])).reshape(-1,))
    W1_errors = np.append(W1_errors,W1_loop)
    # Get Means
    Mu_hat = np.sum((Predicted_Weights[x_i])*(Barycenters_Array))
    Mu = np.mean(np.array(Y_train_locations[x_i]))
    Mean_errors =  np.append(Mean_errors,(Mu_hat-Mu))
    ## Update Erros
    predictions_mean = np.append(predictions_mean,Mu_hat)
    true_mean = np.append(true_mean,Mu)
    # Get Var (non-centered)
    Var_hat = np.sum((Barycenters_Array**2)*(Predicted_Weights[x_i]))
    Var = np.mean(np.array(Y_train_locations[x_i])**2)
    Var_errors = np.append(Var_errors,(Var_hat-Var)**2)
    # Get skewness (non-centered)
    Skewness_hat = np.sum((Barycenters_Array**3)*(Predicted_Weights[x_i]))
    Skewness = np.mean(np.array(Y_train_locations[x_i])**3)
    Skewness_errors = np.append(Skewness_errors,(abs(Skewness_hat-Skewness))**(1/3))
    # Get skewness (non-centered)
    Kurtosis_hat = np.sum((Barycenters_Array**4)*(Predicted_Weights[x_i]))
    Kurtosis = np.mean(np.array(Y_train_locations[x_i])**4)
    Kurtosis_errors = np.append(Kurtosis_errors,(abs(Kurtosis_hat-Kurtosis))**.25)
    
#---------------------------------------------------------------------------------------------#
W1_95 = bootstrap(W1_errors, n=1000, func=np.mean)(.95)
W1_99 = bootstrap(W1_errors, n=1000, func=np.mean)(.99)
M_95 = bootstrap(predictions_mean, n=1000, func=np.mean)(.95)
M_99 = bootstrap(predictions_mean, n=1000, func=np.mean)(.99)
M_95_MC = bootstrap(true_mean, n=1000, func=np.mean)(.95)
M_99_MC = bootstrap(true_mean, n=1000, func=np.mean)(.99)
#---------------------------------------------------------------------------------------------#
# Compute Error Statistics/Descriptors
W1_Performance = np.array([np.min(np.abs(W1_errors)),np.mean(np.abs(W1_errors)),np.max(np.abs(W1_errors))])
Mean_prediction_Performance = np.array([np.min(np.abs(Mean_errors)),np.mean(np.abs(Mean_errors)),np.max(np.abs(Mean_errors))])
Var_prediction_Performance = np.array([np.min(np.abs(Var_errors)),np.mean(np.abs(Var_errors)),np.max(np.abs(Var_errors))])
Skewness_prediction_Performance = np.array([np.min(np.abs(Skewness_errors)),np.mean(np.abs(Skewness_errors)),np.max(np.abs(Skewness_errors))])
Kurtosis_prediction_Performance = np.array([np.min(np.abs(Kurtosis_errors)),np.mean(np.abs(Kurtosis_errors)),np.max(np.abs(Kurtosis_errors))])

Type_A_Prediction = pd.DataFrame({"W1":W1_Performance,
                                  "E[X']-E[X]":Mean_prediction_Performance,
                                  "(E[X'^2]-E[X^2])^.5":Var_prediction_Performance,
                                  "(E[X'^3]-E[X^3])^(1/3)":Skewness_prediction_Performance,
                                  "(E[X'^4]-E[X^4])^.25":Kurtosis_prediction_Performance},index=["Min","MAE","Max"])
Type_A_Predictions_and_confidence = pd.DataFrame({"W1_99_Train":W1_95,
                                                  "W1error_99_Train":W1_99,
                                                  "M_95_Train":M_95,
                                                  "M_99_Train":M_99,
                                                  "MC_95_Train":M_95_MC,
                                                  "MC_99_Train":M_99_MC},index=["CL","Mean","CU"])


# Write Performance
pd.set_option('display.float_format', '{:.4E}'.format)
Type_A_Prediction.to_latex((results_tables_path+str("Roughness_")+str(Rougness)+"Latent_Width_NSDE"+str(width)+"Problemdimension"+str(problem_dim)+"__TypeAPrediction_Train.tex"))
pd.set_option('display.float_format', '{:.4E}'.format)
(Type_A_Predictions_and_confidence.T).to_latex((results_tables_path+str("Roughness_")+str(Rougness)+"Latent_Width_NSDE"+str(width)+"Problemdimension"+str(problem_dim)+"__TypeAPrediction_Train_predictions_w_confidence_intervals.tex"))

# #---------------------------------------------------------------------------------------------#
# # Update User
# Type_A_Prediction

  0%|          | 0/100 [00:00<?, ?it/s]

Building Training Set Performance Metrics


100%|██████████| 100/100 [00:01<00:00, 70.61it/s]


---

### Test-Set Result(s): 

In [24]:
print("Building Test Set Performance Metrics")

# Initialize Wasserstein-1 Error Distribution
W1_errors_test = np.array([])
Mean_errors_test = np.array([])
Var_errors_test = np.array([])
Skewness_errors_test = np.array([])
Kurtosis_errors_test = np.array([])
# Initialize Prediction Metrics
predictions_mean_test = np.array([])
true_mean_test = np.array([])
#---------------------------------------------------------------------------------------------#

# Populate Error Distribution
for x_i in tqdm(range(X_test.shape[0])):    
    # Get Laws
    W1_loop_test = ot.emd2_1d(Barycenters_Array,
                         np.array(Y_test_locations[x_i]).reshape(-1,),
                         Predicted_Weights_test[x_i,].reshape(-1,),
                         (np.array(Y_test_locations[x_i])).reshape(-1,))
    W1_errors_test = np.append(W1_errors_test,W1_loop_test)
    # Get Means
    Mu_hat_test = np.sum((Predicted_Weights_test[x_i])*(Barycenters_Array))
    Mu_test = np.mean(np.array(Y_test_locations[x_i]))
    Mean_errors_test = np.append(Mean_errors_test,(Mu_hat_test-Mu_test))
    ## Update Predictions
    predictions_mean_test = np.append(predictions_mean_test,Mu_hat_test)
    true_mean_test = np.append(true_mean_test,Mu_test)
    # Get Var (non-centered)
    Var_hat_test = np.sum((Barycenters_Array**2)*(Predicted_Weights_test[x_i]))
    Var_test = np.mean(np.array(Y_test_locations[x_i])**2)
    Var_errors_test = np.append(Var_errors_test,(Var_hat_test-Var_test)**2)
    # Get skewness (non-centered)
    Skewness_hat_test = np.sum((Barycenters_Array**3)*(Predicted_Weights_test[x_i]))
    Skewness_test = np.mean(np.array(Y_test_locations[x_i])**3)
    Skewness_errors_test = np.append(Skewness_errors_test,(abs(Skewness_hat_test-Skewness_test))**(1/3))
    # Get skewness (non-centered)
    Kurtosis_hat_test = np.sum((Barycenters_Array**4)*(Predicted_Weights_test[x_i]))
    Kurtosis_test = np.mean(np.array(Y_test_locations[x_i])**4)
    Kurtosis_errors_test = np.append(Kurtosis_errors_test,(abs(Kurtosis_hat_test-Kurtosis_test))**.25)
    
#---------------------------------------------------------------------------------------------#
W1_95_test = bootstrap(W1_errors_test, n=1000, func=np.mean)(.95)
W1_99_test = bootstrap(W1_errors_test, n=1000, func=np.mean)(.99)
M_95_test = bootstrap(predictions_mean_test, n=1000, func=np.mean)(.95)
M_99_test = bootstrap(predictions_mean_test, n=1000, func=np.mean)(.99)
M_95_MC_test = bootstrap(true_mean_test, n=1000, func=np.mean)(.95)
M_99_MC_test = bootstrap(true_mean_test, n=1000, func=np.mean)(.99)
#---------------------------------------------------------------------------------------------#
# Compute Error Statistics/Descriptors
W1_Performance_test = np.array([np.min(np.abs(W1_errors_test)),np.mean(np.abs(W1_errors_test)),np.max(np.abs(W1_errors_test))])
Mean_prediction_Performance_test = np.array([np.min(np.abs(Mean_errors_test)),np.mean(np.abs(Mean_errors_test)),np.max(np.abs(Mean_errors_test))])
Var_prediction_Performance_test = np.array([np.min(np.abs(Var_errors_test)),np.mean(np.abs(Var_errors_test)),np.max(np.abs(Var_errors_test))])
Skewness_prediction_Performance_test = np.array([np.min(np.abs(Skewness_errors_test)),np.mean(np.abs(Skewness_errors_test)),np.max(np.abs(Skewness_errors_test))])
Kurtosis_prediction_Performance_test = np.array([np.min(np.abs(Kurtosis_errors_test)),np.mean(np.abs(Kurtosis_errors_test)),np.max(np.abs(Kurtosis_errors_test))])

Type_A_Prediction_test = pd.DataFrame({"W1":W1_Performance_test,
                                  "E[X']-E[X]":Mean_prediction_Performance_test,
                                  "(E[X'^2]-E[X^2])^.5":Var_prediction_Performance_test,
                                  "(E[X'^3]-E[X^3])^(1/3)":Skewness_prediction_Performance_test,
                                  "(E[X'^4]-E[X^4])^.25":Kurtosis_prediction_Performance_test},index=["Min","MAE","Max"])

Type_A_Predictions_and_confidence_test = pd.DataFrame({"W1_99_Test":W1_95_test,
                                                       "W1error_99_Test":W1_99_test,
                                                       "M_95_Test":M_95_test,
                                                       "M_99_Test":M_99_test,
                                                       "MC_95_Test":M_95_MC_test,
                                                       "MC_99_Test":M_99_MC_test},index=["CL","Mean","CU"])

# Write Performance
pd.set_option('display.float_format', '{:.4E}'.format)
Type_A_Prediction_test.to_latex((results_tables_path+str("Roughness_")+str(Rougness)+"Latent_Width_NSDE"+str(width)+"Problemdimension"+str(problem_dim)+"__TypeAPrediction_Test.tex"))
pd.set_option('display.float_format', '{:.4E}'.format)
(Type_A_Predictions_and_confidence_test.T).to_latex((results_tables_path+str("Roughness_")+str(Rougness)+"Latent_Width_NSDE"+str(width)+"Problemdimension"+str(problem_dim)+"__TypeAPrediction_Test_predictions_w_confidence_intervals.tex"))

 40%|████      | 8/20 [00:00<00:00, 74.47it/s]

Building Test Set Performance Metrics


100%|██████████| 20/20 [00:00<00:00, 70.95it/s]


## Update User

### Print for Terminal Legibility

In [25]:
print("#----------------------#")
print("Training-Set Performance")
print("#----------------------#")
print(Type_A_Prediction)
print(" ")
print(" ")
print(" ")

print("#------------------#")
print("Test-Set Performance")
print("#------------------#")
print(Type_A_Prediction_test)
print(" ")
print(" ")
print(" ")

#----------------------#
Training-Set Performance
#----------------------#
            W1  E[X']-E[X]  (E[X'^2]-E[X^2])^.5  (E[X'^3]-E[X^3])^(1/3)  \
Min 1.5430E+00  3.2919E-05           1.4398E-08              6.9778E-02   
MAE 1.3997E+02  2.8547E+03           9.0365E+07              2.8567E+01   
Max 7.1057E+02  4.0890E+03           2.8011E+08              4.0917E+01   

     (E[X'^4]-E[X^4])^.25  
Min            1.7259E-01  
MAE            1.6067E+01  
Max            2.3011E+01  
 
 
 
#------------------#
Test-Set Performance
#------------------#
            W1  E[X']-E[X]  (E[X'^2]-E[X^2])^.5  (E[X'^3]-E[X^3])^(1/3)  \
Min 1.2226E+01  1.3708E-01           8.0305E-01              1.6369E+00   
MAE 1.3653E+02  2.5211E+03           7.3363E+07              2.5311E+01   
Max 6.2265E+02  4.0013E+03           2.5686E+08              4.0040E+01   

     (E[X'^4]-E[X^4])^.25  
Min            2.0892E+00  
MAE            1.4294E+01  
Max            2.2518E+01  
 
 
 


### Training-Set Performance

In [26]:
Type_A_Prediction

,W1,E[X']-E[X],(E[X'^2]-E[X^2])^.5,(E[X'^3]-E[X^3])^(1/3),(E[X'^4]-E[X^4])^.25
Min,1.5430E+00,3.2919E-05,1.4398E-08,6.9778E-02,1.7259E-01
MAE,1.3997E+02,2.8547E+03,9.0365E+07,2.8567E+01,1.6067E+01
Max,7.1057E+02,4.0890E+03,2.8011E+08,4.0917E+01,2.3011E+01


### Test-Set Performance

In [27]:
Type_A_Prediction_test

,W1,E[X']-E[X],(E[X'^2]-E[X^2])^.5,(E[X'^3]-E[X^3])^(1/3),(E[X'^4]-E[X^4])^.25
Min,1.2226E+01,1.3708E-01,8.0305E-01,1.6369E+00,2.0892E+00
MAE,1.3653E+02,2.5211E+03,7.3363E+07,2.5311E+01,1.4294E+01
Max,6.2265E+02,4.0013E+03,2.5686E+08,4.0040E+01,2.2518E+01


# Model Complexity

In [28]:
Model_Complexity

,N_Centers,N_Q,N_Params,Training Time,T_Test/T_Test-MC,Time Test,Time EM-MC
Model_Complexity_metrics,58,1000,783558,4.6583E+01,1.0736E-02,4.7426E-02,4.4174E+00


---

---
# Fin
---

---